In [2]:
import numpy as np
import time

In [34]:
def SNG_10(seed: int, target: float,
        bitstream_length=1024, bit1=7, bit2=10, isprint=False):

    # check the seed in range (0-1023)
    assert seed > 0 and seed < 1023, "seed should be in range (0-1023)"
    # check the target in range (-1, 1)
    assert target >= -1 and target <= 1, "target should be in range (-1, 1)"
    
    # convert seed to binary
    seed_bin_str = format(seed, '010b')
    seed = np.array([int(i) for i in seed_bin_str])

    # convert target to unipolar
    target = (target + 1) / 2 # (-1, 1) -> (0, 1)
    target_10 = target * 1023

    lfsr = seed.copy()
    output_stream = np.zeros(bitstream_length)

    # do xor on bit1 and bit2
    for i in range(bitstream_length):
        xorbit = lfsr[10-bit1] ^ lfsr[10-bit2]  # xor on bit7 and bit10
        lfsr = np.roll(lfsr, -1)                # shift left
        lfsr[9] = xorbit                        # set xorbit to bit0
        lfsr_val = int(''.join([str(i) for i in lfsr]), 2) # array->binary->int
        if isprint: print(lfsr_val)
        if lfsr_val <= target_10:
            output_stream[i] = 1
        else:
            output_stream[i] = 0

    return output_stream

def SNG_10_matrix(seed_matrix: np.array, target_matrix: np.array,
                  bitstream_length=1024, bit1=7, bit2=10, isprint=False):

    lfsr_matrix = seed_matrix.copy()
    target_matrix = (target_matrix + 1) / 2 * 1023

    output_stream_matrix = np.zeros((seed_matrix.shape[0], seed_matrix.shape[1], bitstream_length), dtype=np.int8)

    # Create weights matrix to convert binary matrix to integers
    weights = np.array([2**i for i in reversed(range(seed_matrix.shape[2]))])
    # if isprint: print('weights: \n',weights) # [512 256 128  64  32  16   8   4   2   1]

    for i in range(bitstream_length):
        xorbits = lfsr_matrix[:,:, 10-bit1] ^ lfsr_matrix[:,:, 10-bit2]
        lfsr_matrix = np.roll(lfsr_matrix, -1, axis=2)
        lfsr_matrix[:,:, 9] = xorbits

        # Matrix multiplication to get integer values
        lfsr_values = (lfsr_matrix * weights).sum(axis=2)

        if isprint: print(lfsr_values)
        
        # Set output_stream_matrix values based on the condition
        output_stream_matrix[:,:, i] = (lfsr_values <= target_matrix).astype(np.int8)

    return output_stream_matrix


def test_methods():
    
    seed_matrix = np.random.randint(0, 2, (28, 28, 10))
    target_matrix = np.random.uniform(0, 1, (28, 28))
    print(seed_matrix[0,0,:])
    print(target_matrix[0,0])

    start = time.time()
    SC_ = SNG_10_matrix(seed_matrix, target_matrix, isprint=False)
    end = time.time()
    print('SNG_10_matrix use :{} seconds'.format(end-start))
    print((SC_[0,0,:]).mean()*2-1)

for i in range(10):
    test_methods()



[1 0 0 1 1 1 0 1 0 0]
0.0
SNG_10_matrix use :0.031011104583740234 seconds
0.0
[0 0 0 0 0 1 1 1 0 1]
0.0
SNG_10_matrix use :0.03100442886352539 seconds
0.0
[1 0 0 0 0 1 1 1 1 0]
0.0
SNG_10_matrix use :0.030996084213256836 seconds
0.0
[1 0 0 1 1 0 1 1 1 1]
0.0
SNG_10_matrix use :0.030001163482666016 seconds
0.0
[1 1 0 1 0 1 0 1 0 0]
0.0
SNG_10_matrix use :0.03000020980834961 seconds
-0.001953125
[1 0 1 0 1 0 1 1 0 0]
0.0
SNG_10_matrix use :0.0299990177154541 seconds
0.0
[0 1 0 1 1 0 1 0 0 1]
0.0
SNG_10_matrix use :0.0310366153717041 seconds
-0.001953125
[0 1 0 1 0 1 0 1 1 1]
0.0
SNG_10_matrix use :0.03296661376953125 seconds
-0.001953125
[0 1 0 0 1 0 0 0 1 0]
0.0
SNG_10_matrix use :0.03168058395385742 seconds
-0.001953125
[1 0 0 1 0 0 1 0 1 0]
0.0
SNG_10_matrix use :0.03095555305480957 seconds
0.0


In [23]:
0.033 * 70000 / 60 # 38.5 minutes

38.5